In [ ]:
import pandas as pd
import numpy as np
from pyspark import sql, SparkContext,  SparkConf
from pyspark.sql import SQLContext
from sklearn.model_selection import train_test_split
from pyspark.sql import Row

In [ ]:
# no need to run sometimes for local or docker use
# conf = SparkConf().setAppName("implicitALS")
# sc = SparkContext(conf=conf)

In [ ]:
sqlC = SQLContext(sc)

In [ ]:
# for local or docker use
# df = sqlC.read.csv('new training list.csv',header=True)

In [ ]:
# need to replace this with s3 link
df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/new_training_list-004ea.csv')

In [ ]:
ratings=df.select("user_id","New_id","srcount").rdd

In [ ]:
# generate unique list of user_id+new_id for predictAll
user=df.select("user_id").distinct()
st=df.select("New_id").distinct()
user_st=user.crossJoin(st)

In [ ]:
# do not convert to rdd first 
# user_st=user_st.rdd

In [ ]:
# only used in local or docker
# df1 = sqlC.read.csv('user-item list.csv',header=True)

In [ ]:
df1_renamed = user_st.withColumnRenamed('user_id', 'user_id1').withColumnRenamed('New_id', 'New_id1')

In [ ]:
# register temp table
df.registerTempTable("df")
df1_renamed.registerTempTable('df1')

In [ ]:
# remove all the right key, do not want all the st that is requested before
removed=sqlC.sql("Select a.* from df1 a left join df b on (a.user_id1=b.user_id and a.New_id1=b.New_id) where (b.New_id is null and b.user_id is null)")

In [ ]:
removed=removed.limit(removed.count())

In [ ]:
removed=removed.rdd

In [ ]:
from pyspark.mllib.recommendation import ALS
# rank is no features
model = ALS.trainImplicit(ratings, rank=20, iterations=10)

In [ ]:
# huge elastic search table with all the service_type and all the confidence score
final_table=model.predictAll(removed)

In [ ]:
# need to push final_tabledf to somewhere
final_tabledf=final_table.toDF()

In [ ]:
final_tabledf.registerTempTable("final_table")

In [ ]:
# testing and querying final table
test=spark.sql("select * from test_final_table order by user desc, rating desc limit 500")
display(test.select('*'))

In [ ]:
# tried huge table with all the user_id but only 6 recommended items
# taking too long
# predicted = []

# for i in user_list:
#     predict=model.recommendProducts(i,6)
#     predicted.append(predict)